In [2]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
words = open('names.txt').read().splitlines()
print(words[:8])
print(len(words))

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']
32033


In [25]:
chars = sorted(set(''.join(words)))
s_to_i = {s:i+1 for i,s in enumerate(chars)}
s_to_i['.'] = 0
i_to_s = {i+1:s for i,s in enumerate(chars)}
i_to_s[0] = '.'

In [ ]:
# i_to_s1 = {i+1:s for i,s in enumerate(chars)}
# i_to_s2 = {i:s for s,i in s_to_i.items()}

# print(i_to_s1)
# print(i_to_s2)
# i_to_s1 == i_to_s2

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z'}
{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z'}


True

Build the dataset

In [115]:
block_size = 3 #context length: number of chars do we take to predict the next one
X, Y = [], []
for w in words:
    # print(w)
    context = [0]*block_size
    for c in w + '.':
        ix = s_to_i[c]
        X.append(context)
        Y.append(ix)
        # print(''.join(i_to_s[i] for i in context), '--->', i_to_s[ix])
        context = context[1:] + [ix]

X = torch.tensor(X)
Y = torch.tensor(Y)

# print(X,Y)

In [ ]:
# block_size = 3 #context length: number of chars do we take to predict the next one
# X, Y = [], []
# for w in words[]:
#     context = [0]*block_size
#     for c in w + '.':
#         ix = s_to_i[c]
#         X.append(context)
#         Y.append(ix)
#         context = context[1:] + [ix]

# X = torch.tensor(X)
# Y = torch.tensor(Y)

In [ ]:
C = torch.randn(27,2) #27 chars 2 dims
C

tensor([[ 0.4604, -0.3841],
        [ 0.1356, -0.6385],
        [-0.2575,  0.1828],
        [-1.1195,  0.0883],
        [ 0.8663, -0.5482],
        [-1.0041,  0.2904],
        [-1.0070, -2.3170],
        [ 0.4037,  0.5119],
        [ 2.4668, -2.0658],
        [ 0.1285, -1.0569],
        [ 0.3842, -1.5807],
        [-1.2396,  0.7150],
        [ 0.5643,  1.1903],
        [-1.3303,  1.4986],
        [ 0.1833, -0.1089],
        [-0.1209,  1.0514],
        [ 0.2814,  0.9989],
        [ 0.4262, -0.1158],
        [ 0.4951, -2.4736],
        [ 0.5132,  1.4697],
        [-1.2437, -0.2826],
        [-0.2650, -0.0033],
        [ 1.0289,  1.8853],
        [ 0.9488, -0.4665],
        [ 0.2649, -0.5232],
        [-0.9316, -1.7019],
        [-0.6356, -0.0966]])

In [ ]:
C[5] #same as F.one_hot(torch.tensor(5), num_classes=27).float() @ C

tensor([-1.0041,  0.2904])

In [98]:
# C[[[(0,0,5), (0,5,1), (0,0,0)]]] #embeddings for ..e, .ea, ... resp
C[torch.tensor([[ 0,  0,  5],
        [ 0,  5, 1],
        [ 0,  0, 0]])]

tensor([[[ 0.4604, -0.3841],
         [ 0.4604, -0.3841],
         [-1.0041,  0.2904]],

        [[ 0.4604, -0.3841],
         [-1.0041,  0.2904],
         [ 0.1356, -0.6385]],

        [[ 0.4604, -0.3841],
         [ 0.4604, -0.3841],
         [ 0.4604, -0.3841]]])

In [ ]:
C[X] #shape is 32,3,2 (32 data points like ..., ..e, .em, emm, etc.; 3 is the context like ...; 2 is the dimension of each character)
#C[5] would be the embedding for e
#C[[]]

tensor([[[ 0.4604, -0.3841],
         [ 0.4604, -0.3841],
         [ 0.4604, -0.3841]],

        [[ 0.4604, -0.3841],
         [ 0.4604, -0.3841],
         [-1.0041,  0.2904]],

        [[ 0.4604, -0.3841],
         [-1.0041,  0.2904],
         [-1.3303,  1.4986]],

        [[-1.0041,  0.2904],
         [-1.3303,  1.4986],
         [-1.3303,  1.4986]],

        [[-1.3303,  1.4986],
         [-1.3303,  1.4986],
         [ 0.1356, -0.6385]],

        [[ 0.4604, -0.3841],
         [ 0.4604, -0.3841],
         [ 0.4604, -0.3841]],

        [[ 0.4604, -0.3841],
         [ 0.4604, -0.3841],
         [-0.1209,  1.0514]],

        [[ 0.4604, -0.3841],
         [-0.1209,  1.0514],
         [ 0.5643,  1.1903]],

        [[-0.1209,  1.0514],
         [ 0.5643,  1.1903],
         [ 0.1285, -1.0569]],

        [[ 0.5643,  1.1903],
         [ 0.1285, -1.0569],
         [ 1.0289,  1.8853]],

        [[ 0.1285, -1.0569],
         [ 1.0289,  1.8853],
         [ 0.1285, -1.0569]],

        [[ 1.0289,  1

In [71]:
emb = C[X]
emb.shape

torch.Size([32, 3, 2])

In [72]:
W1 = torch.rand((6,100)) #6 cuz 3 context * 2 dimensions
b1 = torch.rand(100)

In [92]:
##we need to convert emb into something that we can multiply with W1 because we can't do 32,3,2 * 6,100
##emb[:, 0, :], emb[:, 1, :], emb[:, 2, :] #first char, second char, and third char
##torch.unbind(emb, 1) is the same as emb[:, 0, :], emb[:, 1, :], emb[:, 2, :]
# concat_tensors = torch.unbind(emb, 1) 
# shapes = [concat_tensors[i].shape for i in range(3)]
# print(shapes)
# torch.cat(concat_tensors, 1) #along the first dimension and not the zeroth
#we can do the above or simply just do the following
emb.view(32,6)

tensor([[ 0.4604, -0.3841,  0.4604, -0.3841,  0.4604, -0.3841],
        [ 0.4604, -0.3841,  0.4604, -0.3841, -1.0041,  0.2904],
        [ 0.4604, -0.3841, -1.0041,  0.2904, -1.3303,  1.4986],
        [-1.0041,  0.2904, -1.3303,  1.4986, -1.3303,  1.4986],
        [-1.3303,  1.4986, -1.3303,  1.4986,  0.1356, -0.6385],
        [ 0.4604, -0.3841,  0.4604, -0.3841,  0.4604, -0.3841],
        [ 0.4604, -0.3841,  0.4604, -0.3841, -0.1209,  1.0514],
        [ 0.4604, -0.3841, -0.1209,  1.0514,  0.5643,  1.1903],
        [-0.1209,  1.0514,  0.5643,  1.1903,  0.1285, -1.0569],
        [ 0.5643,  1.1903,  0.1285, -1.0569,  1.0289,  1.8853],
        [ 0.1285, -1.0569,  1.0289,  1.8853,  0.1285, -1.0569],
        [ 1.0289,  1.8853,  0.1285, -1.0569,  0.1356, -0.6385],
        [ 0.4604, -0.3841,  0.4604, -0.3841,  0.4604, -0.3841],
        [ 0.4604, -0.3841,  0.4604, -0.3841,  0.1356, -0.6385],
        [ 0.4604, -0.3841,  0.1356, -0.6385,  1.0289,  1.8853],
        [ 0.1356, -0.6385,  1.0289,  1.8

In [101]:
h = torch.tanh(emb.view(-1,6) @ W1 + b1)
h

tensor([[ 0.6761,  0.5659,  0.6005,  ...,  0.1617,  0.7569,  0.4292],
        [ 0.2660, -0.5561, -0.1659,  ...,  0.2597,  0.4379,  0.0273],
        [ 0.5546, -0.6697, -0.7154,  ...,  0.5363, -0.1715,  0.6781],
        ...,
        [ 0.9947,  0.9982,  0.9891,  ..., -0.2216,  0.9863,  0.8133],
        [ 0.3774,  0.7070,  0.9044,  ...,  0.3235,  0.9578, -0.6207],
        [ 0.4217, -0.4556, -0.2023,  ...,  0.6681, -0.4259, -0.5380]])

In [ ]:
W2 = torch.randn((100,27))
b2 = torch.randn(27)

In [ ]:
logits = h @ W2 + b2
logits.shape

torch.Size([32, 27])

In [104]:
counts = logits.exp()
prob = counts / counts.sum(1, keepdims=True)

In [106]:
#prob[0,5] will tell us what the probability that the NN predicts e is since it is the 5th letter and the 1st target char
#prob[1,13] will tell us what the probability that the NN predicts e is since it is the 13th letter and the 2nd target char
loss = -prob[torch.arange(32), Y].log().mean()
loss

tensor(14.2902)

Cleaned code for the NN

In [116]:
X.shape, Y.shape

(torch.Size([228146, 3]), torch.Size([228146]))

The network architecture is as follows:<br>
6 neurons in the input layer (3 chars * 2 dim)<br>
100 neurons in the 1st hidden layer<br>
27 neurons in the output layer after softmax


In [120]:
g = torch.Generator().manual_seed(2147483647)
C = torch.randn((27,2), generator=g)
W1 = torch.randn((6,100), generator=g)
b1 = torch.randn(100,  generator=g)
W2 = torch.randn(100,27,  generator=g)
b2 = torch.randn(27,  generator=g)
parameters = [C, W1, b1, W2, b2]

print(sum(p.nelement() for p in parameters))

3481


In [ ]:
# for p in parameters:
#     p.requires_grad = True

# for _ in range (1000):
#     # forward pass
#     emb = C[X] #(32,3,2)
#     h = torch.tanh(emb.view(-1,6) @ W1 + b1) #(32,100)
#     logits = h @ W2 + b2 #(32, 27)
#     # counts = logits.exp()
#     # prob = counts / counts.sum(1, keepdims=True)
#     # loss = -prob[torch.arange(32), Y].log().mean()
#     loss = F.cross_entropy(logits, Y) #this is the same as the three lines above
#     print(_,loss.item())

#     #backward pass
#     for p in parameters:
#         p.grad = None
#     loss.backward()

#     #update
#     for p in parameters:
#         p.data += -0.1 * p.grad

0 2.59354567527771
1 2.59336256980896
2 2.5931804180145264
3 2.592999219894409
4 2.59281849861145
5 2.5926382541656494
6 2.592458963394165
7 2.592280387878418
8 2.592102289199829
9 2.5919246673583984
10 2.5917482376098633
11 2.591571807861328
12 2.5913965702056885
13 2.591221570968628
14 2.591047525405884
15 2.5908737182617188
16 2.59070086479187
17 2.590528726577759
18 2.5903568267822266
19 2.5901856422424316
20 2.590015172958374
21 2.589845657348633
22 2.5896761417388916
23 2.5895073413848877
24 2.589339256286621
25 2.5891714096069336
26 2.5890049934387207
27 2.5888383388519287
28 2.588672637939453
29 2.5885074138641357
30 2.5883426666259766
31 2.5881783962249756
32 2.588014841079712
33 2.5878515243530273
34 2.5876893997192383
35 2.5875275135040283
36 2.5873658657073975
37 2.587205410003662
38 2.5870444774627686
39 2.5868849754333496
40 2.5867257118225098
41 2.586566925048828
42 2.5864083766937256
43 2.5862503051757812
44 2.5860931873321533
45 2.5859363079071045
46 2.585780143737793


In [121]:
for p in parameters:
    p.requires_grad = True

In [ ]:
for _ in range (1000):

    #minibatch construct
    ix = torch.randint(0, X.shape[0], (32,))
    
    # forward pass
    emb = C[X[ix]] #(32,3,2)
    h = torch.tanh(emb.view(-1,6) @ W1 + b1) #(32,100)
    logits = h @ W2 + b2 #(32, 27)
    # counts = logits.exp()
    # prob = counts / counts.sum(1, keepdims=True)
    # loss = -prob[torch.arange(32), Y].log().mean()
    loss = F.cross_entropy(logits, Y[ix]) #this is the same as the three lines above
    # print(loss.item())

    #backward pass
    for p in parameters:
        p.grad = None
    loss.backward()

    #update
    for p in parameters:
        p.data += -0.1 * p.grad

print(loss.item())

#this is mini batch stochastic gradient descent

2.663205623626709
